In [1]:
import pandas as pd
import numpy as np
import pathlib
import glob
import os

## Check if trackviews are complete

In [2]:
col_desc = pd.read_excel('../../data/column_descriptors_standardized_021023.xlsx', engine='openpyxl')

In [3]:
col_desc.head(10)

,column_ID,wig_files,control,experimental,column_ID_2,column_ID_SI,num_replicates_control,num_replicates_experimental,meaning,year,...,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46
0,2003A_Sassetti,NaN,NaN,NaN,NaN,2003A_Sassetti,NaN,NaN,NaN,2003.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2003B_Sassetti,NaN,NaN,NaN,NaN,2003B_Sassetti,NaN,NaN,NaN,2003.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2005_Rengarajan,NaN,NaN,NaN,NaN,2005_Rengarajan,NaN,NaN,NaN,2005.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2006_Joshi_GI_1,NaN,NaN,NaN,NaN,2006_Joshi_GI_1,NaN,NaN,NaN,2006.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2006_Joshi_GI_2,NaN,NaN,NaN,NaN,2006_Joshi_GI_2,NaN,NaN,NaN,2006.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2011_Griffin_2,1.0,griffin_glycerol,griffin_cholesterol,griffin_cholesterol_vs_griffin_glycerol,2011_Griffin_2,2.0,3.0,Mutants exhiting altered fitness during choles...,2011.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2012_Zhang,NaN,NaN,NaN,NaN,2012_Zhang,NaN,NaN,Mutants exhibiting altered fitness in growth o...,2012.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2013_DeJesus,1.0,mbio_H37Rv,griffin_glycerol,griffin_glycerol_vs_mbio_H37Rv,NaN,14.0,2.0,NaN,2013.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2013_Zhang_1A,1.0,zhang_input_library,zhang_wt_mouse_d10,zhang_wt_mouse_d10_vs_zhang_input_library,2013_Zhang_1A,1.0,3.0,Mutants exhibiting altered fitness in mouse in...,2013.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2013_Zhang_1B,1.0,zhang_input_library,zhang_wt_mouse_d45,zhang_wt_mouse_d45_vs_zhang_input_library,2013_Zhang_1B,1.0,3.0,Mutants exhibiting altered fitness in mouse in...,2013.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
col_desc.column_ID_SI.unique()

array(['2003A_Sassetti', '2003B_Sassetti', '2005_Rengarajan',
       '2006_Joshi_GI_1', '2006_Joshi_GI_2', '2011_Griffin_2',
       '2012_Zhang', nan, '2013_Zhang_1A', '2013_Zhang_1B',
       '2013_Zhang_2', '2013_Zhang_3A', '2013_Zhang_3B', '2013_Zhang_3C',
       '2013_Zhang_3D', '2015_Kieser_GI_1', '2015_Kieser_GI_2',
       '2015_Kieser_GI_3', '2015_Mendum', '2016_Nambi', '2016_Korte',
       '2017B_DeJesus_1A', '2017B_DeJesus_1B', '2017B_DeJesus_1C',
       '2017_Xu_1A', '2017_Xu_1B', '2017_Xu_1C', '2017_Xu_1D',
       '2017_Xu_1E', '2017_Mishra_1', '2017_Mishra_2', '2018_Carey_1A',
       '2018_Carey_1B', '2018_Carey_1C', '2018_Carey_1D', '2018_Carey_1E',
       '2018_Carey_1F', '2018_Carey_1G', '2018_Carey_1H',
       '2018_Rittershaus_1B', '2018_Rittershaus_1A',
       '129s1.SvImJ_vs_in_vitro', 'A.J_vs_in_vitro',
       'C57BL.6J_vs_in_vitro', 'CC001.Unc_vs_in_vitro',
       'CC002.Unc_vs_in_vitro', 'CC003.Unc_vs_in_vitro',
       'CC004.TauUnc_vs_in_vitro', 'CC005.TauUnc_vs_i

In [7]:
# check if trackviews exist
# trackview_folders = list(pathlib.Path(
#     r'C:\Users\light\Dropbox\tuberculosis\tuberculosis\TrackView').glob('*'))
# trackview_names = [os.path.basename(x) for x in trackview_folders]

In [5]:
# all_conditions = list(col_desc['control']) + list(col_desc['experimental'])

In [6]:
# set(all_conditions).difference(set(trackview_names))

{'nambi_2015_ctpC',
 'nambi_2015_wt',
 'xu_emb_0',
 'xu_emb_2.5',
 'xu_emb_3',
 'xu_inh_0',
 'xu_inh_02',
 'xu_inh_025',
 'xu_mero_0',
 'xu_mero_2.5',
 'xu_rif_0',
 'xu_rif_4',
 'xu_van_0',
 'xu_van_16'}

## make_std_data_dash

In [6]:
col_desc = pd.read_excel('../../data/column_descriptors_standardized_021023.xlsx', engine='openpyxl')
col_desc = col_desc.rename(columns={'column_ID_2': 'column_ID_std'})

In [7]:
lfc = pd.read_csv(
    '../../data/standardized_data/result_logfc_matrix_2023_02_20_CC_BxD.csv')

In [8]:
qval = pd.read_csv(
    '../../data/standardized_data/result_qval_matrix_2023_02_20_CC_BxD.csv')

In [11]:
qval

,Rv_ID,PE35_KO_vs_mbio_H37Rv,PPE68_KO_vs_mbio_H37Rv,Rv0950c_KO_vs_CB_WT,Rv0954_KO_vs_RJ_WT,Rv1096_KO_vs_CB_WT,Rv3005c_KO_day32_vs_dejesus_H37Rv_day32,Rv3594_KO_vs_Rubin_FLUTE_WT,Rv3684_KO_vs_CB_WT,Rv3717_KO_vs_Rubin_FLUTE_WT,...,BXD69_vs_in_vitro_H37Rv,BXD73_vs_in_vitro_H37Rv,BXD73b_vs_in_vitro_H37Rv,BXD77_vs_in_vitro_H37Rv,BXD79_vs_in_vitro_H37Rv,BXD90_vs_in_vitro_H37Rv,BXD93_vs_in_vitro_H37Rv,BXD9_vs_in_vitro_H37Rv,C57BL.6J_vs_in_vitro_H37Rv,DBA.2J_vs_in_vitro_H37Rv
0,Rv0001,1.0,1.0,1.0,1.00000,1.0,0.73174,1.0,1.0,1.00000,...,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.0,1.0,1.00000
1,Rv0002,1.0,1.0,1.0,1.00000,1.0,1.00000,1.0,1.0,1.00000,...,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.0,1.0,1.00000
2,Rv0003,1.0,1.0,1.0,0.43844,1.0,0.47324,1.0,1.0,1.00000,...,0.97187,0.18516,1.00000,1.00000,0.93100,1.00000,1.00000,1.0,1.0,0.91886
3,Rv0004,1.0,1.0,1.0,1.00000,1.0,1.00000,1.0,1.0,1.00000,...,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.0,1.0,0.96831
4,Rv0005,1.0,1.0,1.0,1.00000,1.0,0.62111,1.0,1.0,1.00000,...,1.00000,1.00000,0.98550,1.00000,1.00000,1.00000,1.00000,1.0,1.0,1.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4050,Rv3921c,1.0,1.0,1.0,1.00000,1.0,1.00000,1.0,1.0,1.00000,...,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.0,1.0,1.00000
4051,Rv3922c,1.0,1.0,1.0,1.00000,1.0,0.41371,1.0,1.0,1.00000,...,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.0,1.0,0.92329
4052,Rv3923c,1.0,1.0,1.0,1.00000,1.0,1.00000,1.0,1.0,1.00000,...,0.81113,1.00000,0.60665,0.97738,0.73821,0.35808,0.98407,1.0,1.0,0.22763
4053,Rv3924c,1.0,1.0,1.0,1.00000,1.0,1.00000,1.0,1.0,1.00000,...,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.0,1.0,1.00000


In [10]:
gene_names = pd.read_excel(
    '../../data/annotations/DeJesus_mbio.xlsx', header=1, engine='openpyxl')
gene_names.head()

,ORF ID,Name,Description,Number of TA Sites,Number of Permissive (P) Sites,Number of Non-Permissive (NP) Sites,Number of Sites Belonging to Essential State,Number of Sites Belonging to Growth-Defect State,Number of Sites Belonging to Non-Essential State,Number of Sites Belonging to Growth-Advantage State,Fraction of Sites with Insertions,Mean Normalized Read-Count At Non-Zero Sites,Final Call
0,Rv0001,dnaA,Chromosomal replication initiator protein DnaA,32,29,3,31,0,1,0,0.125000,45.000000,ES
1,Rv0002,dnaN,DNA polymerase III (beta chain) DnaN (DNA nucl...,31,27,4,31,0,0,0,0.032258,1.000000,ES
2,Rv0003,recF,DNA replication and repair protein RecF (singl...,35,32,3,0,0,35,0,0.857143,38.033333,NE
3,Rv0004,-,hypothetical protein,8,8,0,0,0,8,0,0.375000,144.000000,NE
4,Rv0005,gyrB,DNA gyrase (subunit B) GyrB (DNA topoisomerase...,40,36,4,39,0,1,0,0.025000,26.000000,ES


In [12]:
gene_names = gene_names.rename(
    columns={'Name': 'gene_name', 'ORF ID': 'Rv_ID'})

In [13]:
set(lfc.columns).difference(set(qval.columns))

set()

In [14]:
set(qval.columns).difference(set(lfc.columns))

set()

In [15]:
col_ID_standardized = list(col_desc.column_ID_std.unique())

In [16]:
set(qval.columns).difference(set(col_ID_standardized))

{'Rv_ID'}

In [17]:
set(col_ID_standardized).difference(set(qval.columns))

{nan}

In [18]:
lfc.head()

,Rv_ID,PE35_KO_vs_mbio_H37Rv,PPE68_KO_vs_mbio_H37Rv,Rv0950c_KO_vs_CB_WT,Rv0954_KO_vs_RJ_WT,Rv1096_KO_vs_CB_WT,Rv3005c_KO_day32_vs_dejesus_H37Rv_day32,Rv3594_KO_vs_Rubin_FLUTE_WT,Rv3684_KO_vs_CB_WT,Rv3717_KO_vs_Rubin_FLUTE_WT,...,BXD69_vs_in_vitro_H37Rv,BXD73_vs_in_vitro_H37Rv,BXD73b_vs_in_vitro_H37Rv,BXD77_vs_in_vitro_H37Rv,BXD79_vs_in_vitro_H37Rv,BXD9_vs_in_vitro_H37Rv,BXD90_vs_in_vitro_H37Rv,BXD93_vs_in_vitro_H37Rv,C57BL.6J_vs_in_vitro_H37Rv,DBA.2J_vs_in_vitro_H37Rv
0,Rv0001,-0.41,0.01,0.00,0.00,0.00,3.12,-0.06,0.00,-0.06,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,Rv0002,3.28,2.38,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.48,0.00,0.00,0.00
2,Rv0003,-0.03,-0.08,0.03,-1.20,0.39,3.83,0.31,0.17,-0.62,...,0.59,0.97,-0.19,0.05,0.59,0.04,0.13,0.33,0.43,0.32
3,Rv0004,-3.81,-3.39,-2.60,-6.83,1.54,0.00,1.79,1.08,4.91,...,-0.90,-0.90,-0.90,-0.90,-0.90,-0.28,3.95,-0.90,-0.90,-0.90
4,Rv0005,-0.22,0.14,0.13,-0.08,0.27,-2.38,0.70,-0.71,-0.37,...,-0.05,-0.30,-1.79,-1.34,-0.33,-1.84,-0.30,0.71,-0.27,-0.02


In [19]:
lfc = lfc.merge(gene_names[['Rv_ID', 'gene_name', 'Description']], how='left')

In [20]:
lfc = lfc.melt(id_vars=['Rv_ID', 'gene_name', 'Description'],
               var_name='Expt', value_name='log2FC')
qval = qval.melt(id_vars=['Rv_ID'], var_name='Expt', value_name='q-val')

In [21]:
print(lfc.shape, qval.shape)

(592030, 5) (592030, 3)


In [22]:
lfc.head()

,Rv_ID,gene_name,Description,Expt,log2FC
0,Rv0001,dnaA,Chromosomal replication initiator protein DnaA,PE35_KO_vs_mbio_H37Rv,-0.41
1,Rv0002,dnaN,DNA polymerase III (beta chain) DnaN (DNA nucl...,PE35_KO_vs_mbio_H37Rv,3.28
2,Rv0003,recF,DNA replication and repair protein RecF (singl...,PE35_KO_vs_mbio_H37Rv,-0.03
3,Rv0004,-,hypothetical protein,PE35_KO_vs_mbio_H37Rv,-3.81
4,Rv0005,gyrB,DNA gyrase (subunit B) GyrB (DNA topoisomerase...,PE35_KO_vs_mbio_H37Rv,-0.22


In [23]:
std_data = lfc.merge(qval, how='left', on=['Rv_ID', 'Expt'])

In [24]:
std_data.head()

,Rv_ID,gene_name,Description,Expt,log2FC,q-val
0,Rv0001,dnaA,Chromosomal replication initiator protein DnaA,PE35_KO_vs_mbio_H37Rv,-0.41,1.0
1,Rv0002,dnaN,DNA polymerase III (beta chain) DnaN (DNA nucl...,PE35_KO_vs_mbio_H37Rv,3.28,1.0
2,Rv0003,recF,DNA replication and repair protein RecF (singl...,PE35_KO_vs_mbio_H37Rv,-0.03,1.0
3,Rv0004,-,hypothetical protein,PE35_KO_vs_mbio_H37Rv,-3.81,1.0
4,Rv0005,gyrB,DNA gyrase (subunit B) GyrB (DNA topoisomerase...,PE35_KO_vs_mbio_H37Rv,-0.22,1.0


In [25]:
std_data.isna().sum()

Rv_ID             0
gene_name      5694
Description    5694
Expt              0
log2FC         9122
q-val          9122
dtype: int64

In [26]:
std_data['gene_name'] = std_data.apply(
    lambda x: '-' if pd.isna(x.gene_name) else x.gene_name, axis=1)

In [27]:
std_data

,Rv_ID,gene_name,Description,Expt,log2FC,q-val
0,Rv0001,dnaA,Chromosomal replication initiator protein DnaA,PE35_KO_vs_mbio_H37Rv,-0.41,1.00000
1,Rv0002,dnaN,DNA polymerase III (beta chain) DnaN (DNA nucl...,PE35_KO_vs_mbio_H37Rv,3.28,1.00000
2,Rv0003,recF,DNA replication and repair protein RecF (singl...,PE35_KO_vs_mbio_H37Rv,-0.03,1.00000
3,Rv0004,-,hypothetical protein,PE35_KO_vs_mbio_H37Rv,-3.81,1.00000
4,Rv0005,gyrB,DNA gyrase (subunit B) GyrB (DNA topoisomerase...,PE35_KO_vs_mbio_H37Rv,-0.22,1.00000
...,...,...,...,...,...,...
592025,Rv3921c,-,Probable conserved transmembrane protein,DBA.2J_vs_in_vitro_H37Rv,0.00,1.00000
592026,Rv3922c,-,Possible hemolysin,DBA.2J_vs_in_vitro_H37Rv,-2.72,0.92329
592027,Rv3923c,rnpA,Ribonuclease P protein component RnpA (RNaseP ...,DBA.2J_vs_in_vitro_H37Rv,2.24,0.22763
592028,Rv3924c,rpmH,50S ribosomal protein L34 RpmH,DBA.2J_vs_in_vitro_H37Rv,0.00,1.00000


In [29]:
std_data.isna().sum()

Rv_ID             0
gene_name         0
Description    5694
Expt              0
log2FC         9122
q-val          9122
dtype: int64

In [30]:
std_data[std_data['log2FC'].isna()]

,Rv_ID,gene_name,Description,Expt,log2FC,q-val
61,Rv0061c,-,Hypothetical protein,PE35_KO_vs_mbio_H37Rv,NaN,NaN
65,Rv0064A,vapB1,Possible antitoxin VapB1,PE35_KO_vs_mbio_H37Rv,NaN,NaN
81,Rv0078B,-,hypothetical protein,PE35_KO_vs_mbio_H37Rv,NaN,NaN
161,Rv0157A,-,hypothetical protein,PE35_KO_vs_mbio_H37Rv,NaN,NaN
191,Rv0186Ac,-,NaN,PE35_KO_vs_mbio_H37Rv,NaN,NaN
...,...,...,...,...,...,...
591465,Rv3372A,-,NaN,DBA.2J_vs_in_vitro_H37Rv,NaN,NaN
591761,Rv3666B,-,NaN,DBA.2J_vs_in_vitro_H37Rv,NaN,NaN
591794,Rv3697A,vapB48,Possible antitoxin VapB48,DBA.2J_vs_in_vitro_H37Rv,NaN,NaN
591849,Rv3749Ac,-,NaN,DBA.2J_vs_in_vitro_H37Rv,NaN,NaN


In [31]:
std_data = std_data[~std_data['log2FC'].isna()]

In [32]:
std_data.isna().sum()

Rv_ID            0
gene_name        0
Description    978
Expt             0
log2FC           0
q-val            0
dtype: int64

In [33]:
def discretize_q_values(q_val):
    if pd.isna(q_val):
        return np.nan
    if q_val < 0.01:
        q_val_d = 3
    elif q_val < 0.05:
        q_val_d = 2
    else:
        q_val_d = 1
    return q_val_d

In [34]:
std_data['q_val_D'] = std_data['q-val'].apply(discretize_q_values)

/Users/anishazaveri/opt/anaconda3/envs/mtbtndb/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [35]:
std_data.head()

,Rv_ID,gene_name,Description,Expt,log2FC,q-val,q_val_D
0,Rv0001,dnaA,Chromosomal replication initiator protein DnaA,PE35_KO_vs_mbio_H37Rv,-0.41,1.0,1
1,Rv0002,dnaN,DNA polymerase III (beta chain) DnaN (DNA nucl...,PE35_KO_vs_mbio_H37Rv,3.28,1.0,1
2,Rv0003,recF,DNA replication and repair protein RecF (singl...,PE35_KO_vs_mbio_H37Rv,-0.03,1.0,1
3,Rv0004,-,hypothetical protein,PE35_KO_vs_mbio_H37Rv,-3.81,1.0,1
4,Rv0005,gyrB,DNA gyrase (subunit B) GyrB (DNA topoisomerase...,PE35_KO_vs_mbio_H37Rv,-0.22,1.0,1


In [36]:
df_uk = pd.read_csv(
    '../../data/annotations/uniprot_annotation_scores.csv')

In [37]:
df_uk.head()

,Rv_ID,Annotation
0,Rv0001,4
1,Rv0002,4
2,Rv0003,2
3,Rv0004,1
4,Rv0005,5


In [38]:
std_data = std_data.merge(
    df_uk, how='left', on='Rv_ID').rename(columns={'Annotation': 'annotation_score'})

In [39]:
std_data.isna().sum()

Rv_ID                  0
gene_name              0
Description          978
Expt                   0
log2FC                 0
q-val                  0
q_val_D                0
annotation_score    7792
dtype: int64

In [40]:
# have to save as tsv because descriptions have commas
std_data.to_csv('../../data/standardized_data_dash.tsv', index=False, sep='\t')

## Make SI data

In [98]:
%reset -f

In [99]:
import pandas as pd
import numpy as np
import pathlib
import glob
import os

In [100]:
col_desc = pd.read_excel('../../data/column_descriptors_standardized_021023.xlsx', engine='openpyxl')

In [101]:
col_desc.head(10)

,column_ID,wig_files,control,experimental,column_ID_2,column_ID_SI,num_replicates_control,num_replicates_experimental,meaning,year,...,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46
0,2003A_Sassetti,NaN,NaN,NaN,NaN,2003A_Sassetti,NaN,NaN,NaN,2003.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2003B_Sassetti,NaN,NaN,NaN,NaN,2003B_Sassetti,NaN,NaN,NaN,2003.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2005_Rengarajan,NaN,NaN,NaN,NaN,2005_Rengarajan,NaN,NaN,NaN,2005.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2006_Joshi_GI_1,NaN,NaN,NaN,NaN,2006_Joshi_GI_1,NaN,NaN,NaN,2006.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2006_Joshi_GI_2,NaN,NaN,NaN,NaN,2006_Joshi_GI_2,NaN,NaN,NaN,2006.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2011_Griffin_2,1.0,griffin_glycerol,griffin_cholesterol,griffin_cholesterol_vs_griffin_glycerol,2011_Griffin_2,2.0,3.0,Mutants exhiting altered fitness during choles...,2011.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2012_Zhang,NaN,NaN,NaN,NaN,2012_Zhang,NaN,NaN,Mutants exhibiting altered fitness in growth o...,2012.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2013_DeJesus,1.0,mbio_H37Rv,griffin_glycerol,griffin_glycerol_vs_mbio_H37Rv,NaN,14.0,2.0,NaN,2013.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2013_Zhang_1A,1.0,zhang_input_library,zhang_wt_mouse_d10,zhang_wt_mouse_d10_vs_zhang_input_library,2013_Zhang_1A,1.0,3.0,Mutants exhibiting altered fitness in mouse in...,2013.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2013_Zhang_1B,1.0,zhang_input_library,zhang_wt_mouse_d45,zhang_wt_mouse_d45_vs_zhang_input_library,2013_Zhang_1B,1.0,3.0,Mutants exhibiting altered fitness in mouse in...,2013.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
col_desc = col_desc.rename(columns={'column_ID_2': 'column_ID_std'})

In [103]:
lfc_si = pd.read_csv(
    '../../data/SI_datasets/SI_log2FC.csv')
lfc_si = lfc_si.drop(columns='gene_name')

In [104]:
qval_si = pd.read_csv(
    '../../data/SI_datasets/SI_qval.csv')
qval_si = qval_si.drop(columns='gene_name')

In [105]:
print(lfc_si.shape, qval_si.shape)

(3990, 41) (3990, 41)


In [106]:
lfc_si

,Rv_ID,2003A_Sassetti,2003B_Sassetti,2005_Rengarajan,2006_Joshi_GI_1,2006_Joshi_GI_2,2011_Griffin_2,2013_Zhang_1A,2013_Zhang_1B,2013_Zhang_2,...,2018_Carey_1B,2018_Carey_1C,2018_Carey_1D,2018_Carey_1E,2018_Carey_1F,2018_Carey_1G,2018_Carey_1H,2017B_DeJesus_1A,2017B_DeJesus_1B,2017B_DeJesus_1C
0,Rv0001,-3.378401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.00,-0.05,4.53,0.00,0.00,2.14,0.00,-0.60,-1.02,-0.72
1,Rv0002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.60,-1.00,-0.72
2,Rv0003,NaN,NaN,NaN,NaN,NaN,NaN,-6.211402,-3.223423,NaN,...,0.03,-0.64,-0.49,-0.09,-0.31,0.03,-0.20,-0.41,0.45,-0.37
3,Rv0004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.51,1.53,1.53,1.52,1.53,2.07,1.51,-0.59,-1.01,-0.72
4,Rv0005,-5.919522,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.14,2.21,1.96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3985,Rv3920c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.78,1.77,2.26,1.79,2.22,1.23,3.08,-0.48,-0.58,-0.68
3986,Rv3921c,-2.974706,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.00,0.00,0.00,0.00,0.00,0.12,0.00,-0.60,-1.01,-0.73
3987,Rv3922c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.02,1.99,0.90,0.78,1.54,-0.79,0.39,1.95,0.32,0.01
3988,Rv3923c,-2.528826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.58,-1.00,-0.70


In [107]:
gene_names = pd.read_excel(
    '../../data/annotations/DeJesus_mbio.xlsx', header=1, engine='openpyxl')
gene_names.head()

,ORF ID,Name,Description,Number of TA Sites,Number of Permissive (P) Sites,Number of Non-Permissive (NP) Sites,Number of Sites Belonging to Essential State,Number of Sites Belonging to Growth-Defect State,Number of Sites Belonging to Non-Essential State,Number of Sites Belonging to Growth-Advantage State,Fraction of Sites with Insertions,Mean Normalized Read-Count At Non-Zero Sites,Final Call
0,Rv0001,dnaA,Chromosomal replication initiator protein DnaA,32,29,3,31,0,1,0,0.125000,45.000000,ES
1,Rv0002,dnaN,DNA polymerase III (beta chain) DnaN (DNA nucl...,31,27,4,31,0,0,0,0.032258,1.000000,ES
2,Rv0003,recF,DNA replication and repair protein RecF (singl...,35,32,3,0,0,35,0,0.857143,38.033333,NE
3,Rv0004,-,hypothetical protein,8,8,0,0,0,8,0,0.375000,144.000000,NE
4,Rv0005,gyrB,DNA gyrase (subunit B) GyrB (DNA topoisomerase...,40,36,4,39,0,1,0,0.025000,26.000000,ES


In [108]:
gene_names = gene_names.rename(
    columns={'Name': 'gene_name', 'ORF ID': 'Rv_ID'})

In [109]:
set(lfc_si.columns).difference(set(qval_si.columns))

set()

In [110]:
set(qval_si.columns).difference(set(lfc_si.columns))

set()

In [111]:
col_ID_SI = list(col_desc.column_ID_SI.unique())

In [112]:
set(qval_si.columns).difference(set(col_ID_SI))

{'Rv_ID'}

In [113]:
# fix this in metadata!
set(col_ID_SI).difference(set(qval_si.columns))

{'129s1.SvImJ_vs_in_vitro',
 'A.J_vs_in_vitro',
 'BXD102_vs_in_vitro_H37Rv',
 'BXD29_vs_in_vitro_H37Rv',
 'BXD39_vs_in_vitro_H37Rv',
 'BXD40_vs_in_vitro_H37Rv',
 'BXD48a_vs_in_vitro_H37Rv',
 'BXD51_vs_in_vitro_H37Rv',
 'BXD54_vs_in_vitro_H37Rv',
 'BXD56_vs_in_vitro_H37Rv',
 'BXD60_vs_in_vitro_H37Rv',
 'BXD62_vs_in_vitro_H37Rv',
 'BXD67_vs_in_vitro_H37Rv',
 'BXD69_vs_in_vitro_H37Rv',
 'BXD73_vs_in_vitro_H37Rv',
 'BXD73b_vs_in_vitro_H37Rv',
 'BXD77_vs_in_vitro_H37Rv',
 'BXD79_vs_in_vitro_H37Rv',
 'BXD90_vs_in_vitro_H37Rv',
 'BXD93_vs_in_vitro_H37Rv',
 'BXD9_vs_in_vitro_H37Rv',
 'C57BL.6J_vs_in_vitro',
 'C57BL.6J_vs_in_vitro_H37Rv',
 'CC001.Unc_vs_in_vitro',
 'CC002.Unc_vs_in_vitro',
 'CC003.Unc_vs_in_vitro',
 'CC004.TauUnc_vs_in_vitro',
 'CC005.TauUnc_vs_in_vitro',
 'CC006.TauUnc_vs_in_vitro',
 'CC007.Unc_vs_in_vitro',
 'CC008.GeniUnc_vs_in_vitro',
 'CC009.Unc_vs_in_vitro',
 'CC010.GeniUnc_vs_in_vitro',
 'CC011.Unc_vs_in_vitro',
 'CC012.GeniUnc_vs_in_vitro',
 'CC013.GeniUnc_vs_in_vitro',

In [75]:
lfc_si.head()

,Rv_ID,2003A_Sassetti,2003B_Sassetti,2005_Rengarajan,2006_Joshi_GI_1,2006_Joshi_GI_2,2011_Griffin_2,2013_Zhang_1A,2013_Zhang_1B,2013_Zhang_2,...,2018_Carey_1B,2018_Carey_1C,2018_Carey_1D,2018_Carey_1E,2018_Carey_1F,2018_Carey_1G,2018_Carey_1H,2017B_DeJesus_1A,2017B_DeJesus_1B,2017B_DeJesus_1C
0,Rv0001,-3.378401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.00,-0.05,4.53,0.00,0.00,2.14,0.00,-0.60,-1.02,-0.72
1,Rv0002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.60,-1.00,-0.72
2,Rv0003,NaN,NaN,NaN,NaN,NaN,NaN,-6.211402,-3.223423,NaN,...,0.03,-0.64,-0.49,-0.09,-0.31,0.03,-0.20,-0.41,0.45,-0.37
3,Rv0004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.51,1.53,1.53,1.52,1.53,2.07,1.51,-0.59,-1.01,-0.72
4,Rv0005,-5.919522,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.14,2.21,1.96


In [76]:
lfc_si = lfc_si.merge(
    gene_names[['Rv_ID', 'gene_name', 'Description']], how='left')

In [77]:
lfc_si.shape

(3990, 43)

In [78]:
lfc_si = lfc_si.melt(id_vars=['Rv_ID', 'gene_name', 'Description'],
                     var_name='Expt', value_name='log2FC')
qval_si = qval_si.melt(id_vars=['Rv_ID'], var_name='Expt', value_name='q-val')

In [79]:
print(lfc_si.shape, qval_si.shape)

(159600, 5) (159600, 3)


In [80]:
lfc_si.head()

,Rv_ID,gene_name,Description,Expt,log2FC
0,Rv0001,dnaA,Chromosomal replication initiator protein DnaA,2003A_Sassetti,-3.378401
1,Rv0002,dnaN,DNA polymerase III (beta chain) DnaN (DNA nucl...,2003A_Sassetti,NaN
2,Rv0003,recF,DNA replication and repair protein RecF (singl...,2003A_Sassetti,NaN
3,Rv0004,-,hypothetical protein,2003A_Sassetti,NaN
4,Rv0005,gyrB,DNA gyrase (subunit B) GyrB (DNA topoisomerase...,2003A_Sassetti,-5.919522


In [81]:
si_data = lfc_si.merge(qval_si, how='left', on=['Rv_ID', 'Expt'])

In [82]:
si_data.head()

,Rv_ID,gene_name,Description,Expt,log2FC,q-val
0,Rv0001,dnaA,Chromosomal replication initiator protein DnaA,2003A_Sassetti,-3.378401,0.004064
1,Rv0002,dnaN,DNA polymerase III (beta chain) DnaN (DNA nucl...,2003A_Sassetti,NaN,NaN
2,Rv0003,recF,DNA replication and repair protein RecF (singl...,2003A_Sassetti,NaN,NaN
3,Rv0004,-,hypothetical protein,2003A_Sassetti,NaN,NaN
4,Rv0005,gyrB,DNA gyrase (subunit B) GyrB (DNA topoisomerase...,2003A_Sassetti,-5.919522,0.000004


In [84]:
si_data.isna().sum()

Rv_ID              0
gene_name        200
Description      200
Expt               0
log2FC         75864
q-val          71392
dtype: int64

In [85]:
si_data['gene_name'] = si_data.apply(
    lambda x: '-' if pd.isna(x.gene_name) else x.gene_name, axis=1)

In [86]:
si_data

,Rv_ID,gene_name,Description,Expt,log2FC,q-val
0,Rv0001,dnaA,Chromosomal replication initiator protein DnaA,2003A_Sassetti,-3.378401,0.004064
1,Rv0002,dnaN,DNA polymerase III (beta chain) DnaN (DNA nucl...,2003A_Sassetti,NaN,NaN
2,Rv0003,recF,DNA replication and repair protein RecF (singl...,2003A_Sassetti,NaN,NaN
3,Rv0004,-,hypothetical protein,2003A_Sassetti,NaN,NaN
4,Rv0005,gyrB,DNA gyrase (subunit B) GyrB (DNA topoisomerase...,2003A_Sassetti,-5.919522,0.000004
...,...,...,...,...,...,...
159595,Rv3920c,-,Conserved protein similar to jag protein,2017B_DeJesus_1C,-0.680000,0.091698
159596,Rv3921c,-,Probable conserved transmembrane protein,2017B_DeJesus_1C,-0.730000,0.098408
159597,Rv3922c,-,Possible hemolysin,2017B_DeJesus_1C,0.010000,0.125413
159598,Rv3923c,rnpA,Ribonuclease P protein component RnpA (RNaseP ...,2017B_DeJesus_1C,-0.700000,0.086522


In [88]:
si_data.dtypes
#si_data['q-val'] = si_data['q-val'].replace({'no replicates':np.nan})

Rv_ID           object
gene_name       object
Description     object
Expt            object
log2FC         float64
q-val          float64
dtype: object

In [89]:
si_data.isna().sum()

Rv_ID              0
gene_name          0
Description      200
Expt               0
log2FC         75864
q-val          71392
dtype: int64

In [90]:
def discretize_q_values(q_val):
    if pd.isna(q_val):
        return np.nan
    if q_val < 0.01:
        q_val_d = 3
    elif q_val < 0.05:
        q_val_d = 2
    else:
        q_val_d = 1
    return q_val_d

In [91]:
si_data['q_val_D'] = si_data['q-val'].apply(discretize_q_values)

In [92]:
si_data.head()

,Rv_ID,gene_name,Description,Expt,log2FC,q-val,q_val_D
0,Rv0001,dnaA,Chromosomal replication initiator protein DnaA,2003A_Sassetti,-3.378401,0.004064,3.0
1,Rv0002,dnaN,DNA polymerase III (beta chain) DnaN (DNA nucl...,2003A_Sassetti,NaN,NaN,NaN
2,Rv0003,recF,DNA replication and repair protein RecF (singl...,2003A_Sassetti,NaN,NaN,NaN
3,Rv0004,-,hypothetical protein,2003A_Sassetti,NaN,NaN,NaN
4,Rv0005,gyrB,DNA gyrase (subunit B) GyrB (DNA topoisomerase...,2003A_Sassetti,-5.919522,0.000004,3.0


In [93]:
df_uk = pd.read_csv(
    '../../data/annotations/uniprot_annotation_scores.csv')
df_uk.head()

,Rv_ID,Annotation
0,Rv0001,4
1,Rv0002,4
2,Rv0003,2
3,Rv0004,1
4,Rv0005,5


In [94]:
si_data = si_data.merge(df_uk, how='left', on='Rv_ID').rename(columns={'Annotation': 'annotation_score'})

In [95]:
si_data.isna().sum()

Rv_ID                   0
gene_name               0
Description           200
Expt                    0
log2FC              75935
q-val               71458
q_val_D             71458
annotation_score     2080
dtype: int64

In [96]:
si_data.to_csv('../../data/si_data_dash.tsv', index=False, sep='\t')

## Make expt metadata

In [117]:
%reset -f

In [118]:
si_missing = {'129s1.SvImJ_vs_in_vitro',
 'A.J_vs_in_vitro',
 'BXD102_vs_in_vitro_H37Rv',
 'BXD29_vs_in_vitro_H37Rv',
 'BXD39_vs_in_vitro_H37Rv',
 'BXD40_vs_in_vitro_H37Rv',
 'BXD48a_vs_in_vitro_H37Rv',
 'BXD51_vs_in_vitro_H37Rv',
 'BXD54_vs_in_vitro_H37Rv',
 'BXD56_vs_in_vitro_H37Rv',
 'BXD60_vs_in_vitro_H37Rv',
 'BXD62_vs_in_vitro_H37Rv',
 'BXD67_vs_in_vitro_H37Rv',
 'BXD69_vs_in_vitro_H37Rv',
 'BXD73_vs_in_vitro_H37Rv',
 'BXD73b_vs_in_vitro_H37Rv',
 'BXD77_vs_in_vitro_H37Rv',
 'BXD79_vs_in_vitro_H37Rv',
 'BXD90_vs_in_vitro_H37Rv',
 'BXD93_vs_in_vitro_H37Rv',
 'BXD9_vs_in_vitro_H37Rv',
 'C57BL.6J_vs_in_vitro',
 'C57BL.6J_vs_in_vitro_H37Rv',
 'CC001.Unc_vs_in_vitro',
 'CC002.Unc_vs_in_vitro',
 'CC003.Unc_vs_in_vitro',
 'CC004.TauUnc_vs_in_vitro',
 'CC005.TauUnc_vs_in_vitro',
 'CC006.TauUnc_vs_in_vitro',
 'CC007.Unc_vs_in_vitro',
 'CC008.GeniUnc_vs_in_vitro',
 'CC009.Unc_vs_in_vitro',
 'CC010.GeniUnc_vs_in_vitro',
 'CC011.Unc_vs_in_vitro',
 'CC012.GeniUnc_vs_in_vitro',
 'CC013.GeniUnc_vs_in_vitro',
 'CC015.Unc_vs_in_vitro',
 'CC016.GeniUnc_vs_in_vitro',
 'CC017.Unc_vs_in_vitro',
 'CC018.Unc_vs_in_vitro',
 'CC019.TauUnc_vs_in_vitro',
 'CC021.Unc_vs_in_vitro',
 'CC022.GeniUnc_vs_in_vitro',
 'CC023.GeniUnc_vs_in_vitro',
 'CC024.GeniUnc_vs_in_vitro',
 'CC025.GeniUnc_vs_in_vitro',
 'CC027.GeniUnc_vs_in_vitro',
 'CC028.GeniUnc_vs_in_vitro',
 'CC030.GeniUnc_vs_in_vitro',
 'CC031.GeniUnc_vs_in_vitro',
 'CC032.GeniUnc_vs_in_vitro',
 'CC033.GeniUnc_vs_in_vitro',
 'CC034.Unc_vs_in_vitro',
 'CC035.Unc_vs_in_vitro',
 'CC037.TauUnc_vs_in_vitro',
 'CC038.GeniUnc_vs_in_vitro',
 'CC039.Unc_vs_in_vitro',
 'CC040.TauUnc_vs_in_vitro',
 'CC041.TauUnc_vs_in_vitro',
 'CC042.GeniUnc_vs_in_vitro',
 'CC043.GeniUnc_vs_in_vitro',
 'CC044.Unc_vs_in_vitro',
 'CC045.GeniUnc_vs_in_vitro',
 'CC046.Unc_vs_in_vitro',
 'CC047.Unc_vs_in_vitro',
 'CC051.TauUnc_vs_in_vitro',
 'CC056.GeniUnc_vs_in_vitro',
 'CC057.Unc_vs_in_vitro',
 'CC059.TauUnc_vs_in_vitro',
 'CC061.GeniUnc_vs_in_vitro',
 'CC065.Unc_vs_in_vitro',
 'Cast.EiJ_vs_in_vitro',
 'DBA.2J_vs_in_vitro_H37Rv',
 'IFNG_vs_in_vitro',
 'NOD.ShiLtJ_vs_in_vitro',
 'NOS_vs_in_vitro',
 'NZO.H1LtJ_vs_in_vitro',
 'PWK.PhJ_vs_in_vitro',
 'Phox_vs_in_vitro',
 'Rag_vs_in_vitro',
 'Wsb.Eij_vs_in_vitro',
 'mbio_H37Rv_vs_in_vitro'}

In [119]:
import pandas as pd
import numpy as np
import pathlib
import glob
import os

In [134]:
col_desc = pd.read_excel('../../data/column_descriptors_standardized_021023.xlsx', engine='openpyxl')

In [135]:
col_desc = col_desc.rename(columns={'column_ID_2': 'column_ID_std'})
col_desc = col_desc.drop(columns=['column_ID', 'wig_files'])
col_desc = col_desc.drop(columns=[c for c in col_desc.columns if 'Unnamed' in c])

In [136]:
col_desc.loc[col_desc['column_ID_SI'].isin(si_missing), 'column_ID_SI'] = np.nan

In [137]:
col_desc.head()

,control,experimental,column_ID_std,column_ID_SI,num_replicates_control,num_replicates_experimental,meaning,year,paper_title,paper_URL,...,carbon_source,stress_description,GI_RvID,GI_name,MicArr_or_TnSeq,stat_analysis,mouse_strain,cell_type,Mtb_strain,plot_SI_graph
0,NaN,NaN,NaN,2003A_Sassetti,NaN,NaN,NaN,2003.0,Genes required for mycobacterial growth define...,https://onlinelibrary.wiley.com/doi/full/10.10...,...,glycerol,-,NaN,NaN,microarray,NaN,NaN,NaN,H37Rv,No
1,NaN,NaN,NaN,2003B_Sassetti,NaN,NaN,NaN,2003.0,Genetic requirements for mycobacterial surviva...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...,...,-,NaN,NaN,NaN,microarray,NaN,NaN,NaN,H37Rv,No
2,NaN,NaN,NaN,2005_Rengarajan,NaN,NaN,NaN,2005.0,Genome-wide requirements for Mycobacterium tub...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,...,-,-,NaN,NaN,microarray,NaN,NaN,NaN,H37Rv,No
3,NaN,NaN,NaN,2006_Joshi_GI_1,NaN,NaN,NaN,2006.0,Characterization of mycobacterial virulence ge...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,...,-,NaN,Rv0173,NaN,microarray,NaN,NaN,NaN,H37Rv,No
4,NaN,NaN,NaN,2006_Joshi_GI_2,NaN,NaN,NaN,2006.0,Characterization of mycobacterial virulence ge...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,...,-,NaN,Rv3502c,NaN,microarray,NaN,NaN,NaN,H37Rv,No


In [138]:
col_desc.isna().sum()

control                         929
experimental                    929
column_ID_std                   929
column_ID_SI                   1035
num_replicates_control          929
num_replicates_experimental     929
meaning                         932
year                            932
paper_title                     932
paper_URL                       953
journal                         932
first_author                    930
last_author                     930
in_vitro_cell_vivo              917
in_vitro_media                 1021
carbon_source                  1020
stress_description             1055
GI_RvID                        1046
GI_name                        1057
MicArr_or_TnSeq                 917
stat_analysis                   959
mouse_strain                    983
cell_type                      1074
Mtb_strain                      917
plot_SI_graph                  1035
dtype: int64

In [139]:
col_desc.drop(columns=['year', 'journal',
                       'first_author', 'last_author', 'control', 'experimental', 'stat_analysis'], inplace=True)

In [140]:
fill_na_cols = [c for c in col_desc.columns if c not in [
    'control', 'experimental', 'column_ID_std', 'column_ID_SI', 'plot_SI_graph']]

In [141]:
col_desc[fill_na_cols] = col_desc[fill_na_cols].fillna('')

In [142]:
col_desc['num_replicates_control'] = col_desc['num_replicates_control'].apply(lambda x: int(x) if x!='' else '')
col_desc['num_replicates_experimental'] = col_desc['num_replicates_experimental'].apply(lambda x: int(x) if x!='' else '')

In [143]:
rename_dict = {
    'num_replicates_control': 'num replicates control',
    'num_replicates_experimental': 'num replicates experimental',
    'in_vitro_cell_vivo': 'in vitro/cell/in vivo',
    'carbon_source': 'carbon source',
    'in_vitro_media': 'in vitro media',
    'stress_description': 'stress',
    'MicArr_or_TnSeq' : 'Microarray or TnSeq',
    'mouse_strain': 'mouse strain',
    'cell_type': 'cell type',
    'Mtb_strain': 'Mtb strain'
}
col_desc = col_desc.rename(columns = rename_dict)

In [144]:
col_desc['stress'] = col_desc['stress'].apply(lambda x: x.replace('_', ' '))

In [145]:
col_desc.to_csv('../../data/col_desc_dash.tsv', index=False, sep='\t')

## Make gene metadata

In [175]:
%reset -f

In [176]:
import pandas as pd
import numpy as np
import pathlib
import glob
import os

In [177]:
si_data = pd.read_csv('../../data/si_data_dash.tsv', sep='\t')
si_data.head()

,Rv_ID,gene_name,Description,Expt,log2FC,q-val,q_val_D,annotation_score
0,Rv0001,dnaA,Chromosomal replication initiator protein DnaA,2003A_Sassetti,-3.378401,0.004064,3.0,4.0
1,Rv0002,dnaN,DNA polymerase III (beta chain) DnaN (DNA nucl...,2003A_Sassetti,NaN,NaN,NaN,4.0
2,Rv0003,recF,DNA replication and repair protein RecF (singl...,2003A_Sassetti,NaN,NaN,NaN,2.0
3,Rv0004,-,hypothetical protein,2003A_Sassetti,NaN,NaN,NaN,1.0
4,Rv0005,gyrB,DNA gyrase (subunit B) GyrB (DNA topoisomerase...,2003A_Sassetti,-5.919522,0.000004,3.0,5.0


In [178]:
std_data = pd.read_csv('../../data/standardized_data_dash.tsv', sep='\t')
std_data.head()

,Rv_ID,gene_name,Description,Expt,log2FC,q-val,q_val_D,annotation_score
0,Rv0001,dnaA,Chromosomal replication initiator protein DnaA,PE35_KO_vs_mbio_H37Rv,-0.41,1.0,1,4.0
1,Rv0002,dnaN,DNA polymerase III (beta chain) DnaN (DNA nucl...,PE35_KO_vs_mbio_H37Rv,3.28,1.0,1,4.0
2,Rv0003,recF,DNA replication and repair protein RecF (singl...,PE35_KO_vs_mbio_H37Rv,-0.03,1.0,1,2.0
3,Rv0004,-,hypothetical protein,PE35_KO_vs_mbio_H37Rv,-3.81,1.0,1,1.0
4,Rv0005,gyrB,DNA gyrase (subunit B) GyrB (DNA topoisomerase...,PE35_KO_vs_mbio_H37Rv,-0.22,1.0,1,5.0


In [179]:
#are all Rvs in si data there in std data?
si_Rvs = set(si_data.Rv_ID.to_list())
std_Rvs = set(std_data.Rv_ID.to_list())


In [180]:
len(si_Rvs)

3990

In [181]:
len(std_Rvs)

4055

In [182]:
si_Rvs.difference(std_Rvs)

set()

In [183]:
si_Rvs.issubset(std_Rvs)

True

In [184]:
# Therefore use std to get gene metadata

In [185]:
gene_metadata = std_data[['Rv_ID', 'gene_name', 'Description']].drop_duplicates()

In [186]:
gene_names = pd.read_excel(
    '../../data/annotations/DeJesus_mbio.xlsx', header=1, engine='openpyxl')
gene_names = gene_names.rename(
    columns={'ORF ID': 'Rv_ID'})
gene_names.head()

,Rv_ID,Name,Description,Number of TA Sites,Number of Permissive (P) Sites,Number of Non-Permissive (NP) Sites,Number of Sites Belonging to Essential State,Number of Sites Belonging to Growth-Defect State,Number of Sites Belonging to Non-Essential State,Number of Sites Belonging to Growth-Advantage State,Fraction of Sites with Insertions,Mean Normalized Read-Count At Non-Zero Sites,Final Call
0,Rv0001,dnaA,Chromosomal replication initiator protein DnaA,32,29,3,31,0,1,0,0.125000,45.000000,ES
1,Rv0002,dnaN,DNA polymerase III (beta chain) DnaN (DNA nucl...,31,27,4,31,0,0,0,0.032258,1.000000,ES
2,Rv0003,recF,DNA replication and repair protein RecF (singl...,35,32,3,0,0,35,0,0.857143,38.033333,NE
3,Rv0004,-,hypothetical protein,8,8,0,0,0,8,0,0.375000,144.000000,NE
4,Rv0005,gyrB,DNA gyrase (subunit B) GyrB (DNA topoisomerase...,40,36,4,39,0,1,0,0.025000,26.000000,ES


In [187]:
gene_metadata = gene_metadata.merge(gene_names[['Rv_ID', 'Final Call']], how='left', on='Rv_ID')

In [188]:
gene_metadata.isna().sum()

Rv_ID           0
gene_name       0
Description    39
Final Call     39
dtype: int64

In [189]:
gene_metadata['Description'] = gene_metadata['Description'].fillna('-')
gene_metadata['Final Call'] = gene_metadata['Final Call'].fillna('Missing')

In [190]:
gene_metadata['Final Call'].value_counts()

NE           3005
ES            461
GA            310
GD            135
Uncertain      76
Missing        39
ESD            29
Name: Final Call, dtype: int64

In [191]:
gene_metadata_rename = {
    'NE': 'Non essential',
    'ES': 'Essential',
    'GA': 'Growth advantage',
    'GD': 'Growth defect',
    'ESD': 'Essential domain'
}
gene_metadata['Final Call'] = gene_metadata['Final Call'].replace(gene_metadata_rename)

In [192]:
gene_metadata

,Rv_ID,gene_name,Description,Final Call
0,Rv0001,dnaA,Chromosomal replication initiator protein DnaA,Essential
1,Rv0002,dnaN,DNA polymerase III (beta chain) DnaN (DNA nucl...,Essential
2,Rv0003,recF,DNA replication and repair protein RecF (singl...,Non essential
3,Rv0004,-,hypothetical protein,Non essential
4,Rv0005,gyrB,DNA gyrase (subunit B) GyrB (DNA topoisomerase...,Essential
...,...,...,...,...
4050,Rv3372A,-,-,Missing
4051,Rv3666B,-,-,Missing
4052,Rv3697A,vapB48,Possible antitoxin VapB48,Non essential
4053,Rv3749Ac,-,-,Missing


In [193]:
func_cats = pd.read_excel('../../data/annotations/H37Rv_tuberculist_functional_categories.xlsx', engine='openpyxl')
func_cats.head()

,Rv_ID,gene_name,category
0,Rv0001,dnaA,information pathways
1,Rv0002,dnaN,information pathways
2,Rv0003,recF,information pathways
3,Rv0004,-,conserved hypotheticals
4,Rv0005,gyrB,information pathways


In [194]:
gene_metadata = gene_metadata.merge(func_cats[['Rv_ID', 'category']], on='Rv_ID', how='left').rename(columns = {'category': 'tuberculist_category'})

In [195]:
gene_metadata['tuberculist_category'] = gene_metadata['tuberculist_category'].fillna('unknown')

In [196]:
gene_metadata['tuberculist_category'].value_counts()

conserved hypotheticals                    1026
intermediary metabolism and respiration     934
cell wall and cell processes                771
lipid metabolism                            271
information pathways                        242
virulence, detoxification, adaptation       220
regulatory proteins                         198
PE/PPE                                      166
insertion seqs and phages                   146
unknown                                      81
Name: tuberculist_category, dtype: int64

In [197]:
umap_data = pd.read_csv('../../data/NN_for_dash.csv')
umap_data.head()

,Rv_ID,NN_1,NN_2,NN_3,NN_4,NN_5,NN_6,NN_7,NN_8,NN_9,NN_10
0,Rv0001,Rv1783,Rv1613,Rv1007c,Rv0952,Rv0283,Rv3795,Rv2992c,Rv2555c,Rv3608c,Rv3834c
1,Rv0002,Rv3596c,Rv2906c,Rv3031,Rv1658,Rv2987c,Rv1652,Rv2109c,Rv1208,Rv2580c,Rv1464
2,Rv0003,Rv3384c,Rv2737c,Rv1082,Rv2200c,Rv3262,Rv1080c,Rv3810,Rv1474c,Rv3919c,Rv1542c
3,Rv0004,Rv3764c,Rv2246,Rv2404c,Rv1248c,Rv3025c,Rv2391,Rv0946c,Rv0066c,Rv1170,Rv1956
4,Rv0005,Rv3518c,Rv1755c,Rv1169c,Rv2870c,Rv2816c,Rv1938,Rv3130c,Rv2428,Rv1848,Rv0220


In [198]:
umap_data_cols = [col for col in umap_data.columns if col!= 'Rv_ID']
umap_data['NN_joined'] = umap_data.apply(lambda x: ', '.join(x[umap_data_cols].values.tolist()), axis=1)

In [199]:
umap_data.head()

,Rv_ID,NN_1,NN_2,NN_3,NN_4,NN_5,NN_6,NN_7,NN_8,NN_9,NN_10,NN_joined
0,Rv0001,Rv1783,Rv1613,Rv1007c,Rv0952,Rv0283,Rv3795,Rv2992c,Rv2555c,Rv3608c,Rv3834c,"Rv1783, Rv1613, Rv1007c, Rv0952, Rv0283, Rv379..."
1,Rv0002,Rv3596c,Rv2906c,Rv3031,Rv1658,Rv2987c,Rv1652,Rv2109c,Rv1208,Rv2580c,Rv1464,"Rv3596c, Rv2906c, Rv3031, Rv1658, Rv2987c, Rv1..."
2,Rv0003,Rv3384c,Rv2737c,Rv1082,Rv2200c,Rv3262,Rv1080c,Rv3810,Rv1474c,Rv3919c,Rv1542c,"Rv3384c, Rv2737c, Rv1082, Rv2200c, Rv3262, Rv1..."
3,Rv0004,Rv3764c,Rv2246,Rv2404c,Rv1248c,Rv3025c,Rv2391,Rv0946c,Rv0066c,Rv1170,Rv1956,"Rv3764c, Rv2246, Rv2404c, Rv1248c, Rv3025c, Rv..."
4,Rv0005,Rv3518c,Rv1755c,Rv1169c,Rv2870c,Rv2816c,Rv1938,Rv3130c,Rv2428,Rv1848,Rv0220,"Rv3518c, Rv1755c, Rv1169c, Rv2870c, Rv2816c, R..."


In [200]:
gene_metadata = gene_metadata.merge(umap_data[['Rv_ID', 'NN_joined']], on='Rv_ID', how='left')

In [201]:
gene_metadata.isna().sum()

Rv_ID                   0
gene_name               0
Description             0
Final Call              0
tuberculist_category    0
NN_joined               0
dtype: int64

In [202]:
gene_metadata.to_csv('../../data/gene_metadata_dash.tsv', index=False, sep='\t')